In [1]:
import sys
import re
import math
import json
import operator
from os import walk
import os

In [2]:
input_path = 'op_spam_training_data'

In [3]:
stop_words = set(open('stopwords.txt', 'r').read().split('\n'))

In [4]:
def parameters():
    with open('nbmodel.txt') as f:
        json_data = json.load(f)
        return json_data

In [5]:
def text_cleaning(text):
    cleaned_txt = re.sub('[^a-z\s]+', ' ', text, flags=re.IGNORECASE)
    return re.sub('(\s+)', ' ', cleaned_txt).lower().replace("'", "")

In [6]:
def category(file_name):
    text = open(file_name, 'r').read()
    
    #Data cleaning and splitting
    cleaned_text = text_cleaning(text)
    word_arr = cleaned_text.split(' ')
    
    # removing stop words
    word_arr = [each for each in word_arr if each not in stop_words]  
    params = parameters()
    prob_class = params['pro_each_class']
    prior_prob = params['probability_score']
    
     # calculate probability of each word
    # count of word in class c +1/ counts of words in whole class + modv + 1
    prob_sentence = {'0': 0,'1': 0 ,'2':0 ,'3':0}
    
    for word in word_arr:
        for key in prob_sentence.keys():
            if word in prior_prob[key]:
                prob_sentence[key] += prior_prob[key][word]

    # posterior Probability
    posterior_prob = {}
    for key in prob_sentence.keys():
        posterior_prob[key] = prob_sentence[key] + prob_class[key]

    return max(posterior_prob.items(), key=operator.itemgetter(1))[0]

In [7]:
def category_names(cat):

    if cat == "0":
        return "deceptive negative"
    elif cat == "1":
        return "truthful negative"
    elif cat == "2":
        return "truthful positive"

    return "deceptive positive"

In [8]:
output = ''
for (dirpath, dirnames, filenames) in walk(next(os.walk(input_path))[0]):
    for file in filenames:
        file_name = os.path.join(dirpath, file)
        search_str ='fold1'
        if bool(re.search(search_str, file_name)):
            cat = category(file_name)
            tag = category_names(cat)
            tag = tag +" "+ file_name + '\n'
            output += tag

with open('nboutput.txt', 'w') as file:
    file.write(str(output))
    file.close()